In [2]:
!pip install ain-py
!pip install nest_asyncio

In [6]:
import asyncio
import nest_asyncio
from time import time
from ain.ain import Ain
from ain.types import ValueOnlyTransactionInput

nest_asyncio.apply()

loop = asyncio.get_event_loop()

ain = Ain("https://testnet-api.ainetwork.ai", chainId=None)
accounts = ain.wallet.create(1)
my_address = accounts[0]
ain.wallet.setDefaultAccount(my_address)

app_name = f"my_app_{int(time()*1000)}"  # Use your own app name
app_path = f"/apps/{app_name}"

# Create an app at /apps/{appName}. With the admin config below,
# only 'my_address' will have owner & write permissions at /apps/{app_name}.
async def create_my_app():
    result = await ain.db.ref(
        f'/manager_app/{app_name}/create/{int(time()*1000)}'
    ).setValue(
        ValueOnlyTransactionInput(
            value={
                'admin': {
                    my_address: True
                }
            },
            nonce=-1
        )
    )
    print(result)
loop.run_until_complete(create_my_app())


async def get_my_app_owner():
    result = await ain.db.ref(app_path).getOwner()
    print(result)
loop.run_until_complete(get_my_app_owner())


async def set_write_rule():
    result = await ain.db.ref(app_path).setRule(
        ValueOnlyTransactionInput(
            value={
                '.rule': {
                    'write': True
                }
            },
            nonce=-1
        )
    )
    print(result)
loop.run_until_complete(set_write_rule())

async def set_value():
    user_message_path = f'{app_path}/messages/{my_address}'
    result = await ain.db.ref(
        f'{user_message_path}/{int(time()*1000)}/user'
    ).setValue(
        ValueOnlyTransactionInput(value='Hello AI Network', nonce=-1)
    )
    print(result)
loop.run_until_complete(set_value())
# loop.close()

{'tx_hash': '0x0556549daac7c9faca14ce1167a652eee98a57ea199a93fb2ca2c354042533ee', 'result': {'gas_amount_total': {'bandwidth': {'service': 1}, 'state': {'service': 0}}, 'gas_cost_total': 0, 'error_message': 'No write permission on: /manager_app/my_app_1641548630933/create/1641548630934', 'code': 103, 'bandwidth_gas_amount': 1, 'gas_amount_charged': 1}}
None
{'tx_hash': '0x73a9d81ad9c54302a505ee5889cbaa7a80973aa000002f59267cd2652dff9a98', 'result': {'gas_amount_total': {'bandwidth': {'service': 0, 'app': {'my_app_1641548630933': 1}}, 'state': {'service': 0}}, 'gas_cost_total': 0, 'error_message': 'No write_rule permission on: /apps/my_app_1641548630933', 'code': 503, 'bandwidth_gas_amount': 1, 'gas_amount_charged': 0}}
{'tx_hash': '0x48943598415ac32db87206354c57b13089f40f18f19df16e35fb2286e436caee', 'result': {'gas_amount_total': {'bandwidth': {'service': 0, 'app': {'my_app_1641548630933': 1}}, 'state': {'service': 0}}, 'gas_cost_total': 0, 'error_message': 'No write permission on: /app